# EMGdecomPy Workflow

This notebook allows for simple visualisation of decomposition results via EMGDecomPy package. For simplicity, you may want to select the usual filepath and filename to store raw EMG signals, and simply "Run All Cells" each time you want to perform the decomposition process.

## Imports - dependencies

Please run the cells below to add all the dependencies required in this notebook.   
First, import the proprietary open-source python packages that were used in building this application:

In [1]:
# loadmat is used to load MATLAB data
from scipy.io import loadmat

# Pickle allows saving and loading python object into a file
import pickle

## Imports - EMGDecomPy

Now, import the proprietary EMGDecomPy package scripts for decomposing and visualizing the electromyography data:

In [2]:
from emgdecompy.decomposition import *
from emgdecompy.contrast import *
from emgdecompy.viz import *
from emgdecompy.preprocessing import *

## Load the raw EMG data

Please specify the filepath where the raw data is located. 

This raw data will be processed via an algorithm built after the process proposed by _Negro et al 2016_ paper. In order for the decomposition to accurately process the data, it needs to be in a clean format. File __extension__ must be __.mat__, the __data type__ must be __dictionary__, and the __key__ containing the raw electromyography signal to be decomposed must be named __'SIG'__.

A simplified example of raw data can be seen below:

Note the aforementioned __'SIG'__ key-value pair, containing array of arrays. For the purposes of decomposition, __'SIG'__ is the only key needed within the dictionary.

In [6]:
raw_data_dict = loadmat('../data/raw/gl_10.mat')

## Run decomposition

Run the decomposition on raw data to retrieve a dictionary containing decomposed results.

__There is a number of parameters you may choose to customize:__

| Parameter    | Data Type                    | Explanation                                                                                  |
|--------------|------------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| x            | numpy.ndarray                | Raw EMG signal. This is expected inside the 'SIG' key within the raw signal loaded above.  
| discard      | slice, int, or array of ints | Indices of channels to discard. Useful if you are looking to only analyze some channels, or expect some channels to contain disruptive data.                               |
| R            | int                          | How far to extend x during the extension step of the algorithm.                                                                                                            |
| M            | int                          | Number of iterations to run decomposition for.                                                                                                                             |
| filter       | bool                         | Whether to band-pass filter the raw EMG signal or not.                                                                                                                     |
| lowcut       | float                        | Lower range of band-pass filter.                                                                                                                                           |
| highcut      | float                        | Upper range of band-pass filter.                                                                                                                                           |
| fs           | float                        | Sampling frequency in Hz.                                                                                                                                                  |
| order        | int                          | Order of band-pass filter.                                                                                                                                                 |
| peel         | bool                         | Whether to conduct "peel-off" process or not.                                                                                                                              |
| Tolx         | float                        | Tolerance for element-wise comparison in separation.                                                                                                                       |
| contrast_fun | function                     | Contrast function to use. skew, og_cosh or exp_sq                                                                                                                          |
| ortho_fun    | function                     | Orthogonalization function to use. gram_schmidt or deflate                                                                                                                 |
| max_iter_sep | int > 0                      | Maximum iterations for fixed point algorithm.                                                                                                                              |
| l            | int                          | Required minimal horizontal distance between peaks in peak-finding algorithm. Default value of 31 samples is approximately equivalent to 15 ms at a 2048 Hz sampling rate. |
| sil_pnr      | bool                         | Whether to use SIL or PNR as acceptance criterion. Default value of True uses SIL.                                                                                         |
| thresh       | float                        | SIL/PNR threshold for accepting a separation vector.                                                                                                                       |
| max_iter_ref | int > 0                      | Maximum iterations for refinement.                                                                                                                                         |
| random_seed  | int                          | Used to initialize the pseudo-random processes in the function.                                                                                                            |
| verbose      | bool                         | If true, decomposition information is printed.                                                                                                                             |


In [7]:
output = decomposition(
    raw_data_dict["SIG"],
    discard=5,
    R=16,
    M=64,
    filter=True,
    lowcut=10,
    highcut = 900,
    fs=2048,
    order=6,
    peel=False,
    Tolx=10e-4,
    contrast_fun=skew,
    ortho_fun=gram_schmidt,
    max_iter_sep=10,
    l=31,
    sil_pnr=True,
    thresh=0.9,
    max_iter_ref=10,
    random_seed=None,
    verbose=False
)

Centred.
Extended.
Whitened.
Extracted source at iteration 4.
Extracted source at iteration 5.
Extracted source at iteration 11.
Extracted source at iteration 25.
Extracted source at iteration 27.
Extracted source at iteration 32.
Extracted source at iteration 35.
Extracted source at iteration 55.
Extracted source at iteration 59.
Extracted source at iteration 63.


The output of decomposition is a dictionary containing the following keys:

__'B'__: Matrix whose columns contain the accepted separation vectors.

__'MUPulses'__: Firing indices for each motor unit. This is the key used within this notebook, along with raw data, for visualisation purposes.

__'SIL'__: Corresponding silhouette scores for each accepted source.

__PNR__: Corresponding pulse-to-noise ratio for each accepted source.

A simplified example of output of decomposed data can be seen below:

## Optional: save the output object

Cell below saves the output of the decomposition.

In [3]:
# decomp_sample = output 
# decomp_sample_pkl = open('decomp_sample_pkl.obj', 'wb') 
# pickle.dump(decomp_sample, decomp_sample_pkl)

Cell below loads the output saved above.

In [4]:
with open('decomp_sample_pkl.obj', 'rb') as f: output = pickle.load(f)

## Visualize results

Finally, run the cells below to get interactive dashboard of the decomposed data.

There are two parameters for the visualize_decomp() function. _decomp\_results_ and _raw_ take in decomposition results and raw signal, respectively. You are unlikely to want to change it within the function, and most likely will select those earlier. However, if you want to plug in the data directly, you may use those parameters.

There are three widgets controlling the rest of the dashboard:   
1. The __Motor Unit__ widget for selecting the Motor Unit to examing
2. The __Preset__ parameter allows to select arrangements of the channel. Currently, you may select between 'standard' and 'vert63' arrangements. New arrangements may easily be added within the channel_preset() function in the viz.py script.
3. The __Method__ widget allows to select the metric for calculating the average mismatch score between peak contribution and average MUAP shape. Currently, RMSE is the only available metric, but other metrics can be added easily in the future by adding functions to the viz.py script.

This will create a simple interactive dashboard. On top of the dashboard is the widget for selecting between motor unit action potentials (MUAPs.) Below the MUAP selection widget is the "Delete Selected Peak" button. Once a specific peak is selected, you may mark it as false positive and it will be removed from the pulse train and rerun the dashboard. Below the widgets, dashboard contains four charts:

#### Chart 1: Signal Strength + Firing Rate. 
The top chart displays signal strength along with the instanteneous firing rate at that point.   
This chart allows for interval control. Select the interval of interest and move it around, and observe the graphs below zoom in according to the selection.

#### Chart 2: Firing Rate. 
The second chart displays the instanteneous firing rate of each spike. Outliars may be indicative of false positive firing. Click on an individual data point to select corresponding pulse. This will allow to see the given firing's contribution to the MUAP shape, and delete it with the click of the button.

#### Chart 3: Signal Strength
The third chart displays signal strength of each firing. Click on an individual data point to select corresponding pulse. This will allow to see the given firing's contribution to the MUAP shape, and delete it with the click of the button.

#### Chart 4: MUAP Shapes + Peak Contributions
The last chart displays the average shape for each channel. The title displays the currently selected motor unit. If an individual peak has been selected using Chart 2 or 3, then an overlay will be displayed on each sub-chart that shows the contribution of the selected peak towards the shape for each channel. You may toggle the opacity of that overlay and the MUAP shape using the legend. Additionally, the peak firing index and RMSE will displayed in the title of the chart.

In [7]:
visualize_decomp(output, raw_data_dict['SIG'])

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

## Save the polished data

After manually removing false positive firing peaks, you may want to save the dictionary containing corrected firings. You can later reload it in the cell above. To save the cleaned up output, please run the cell below.

In [8]:
# decomp_sample = output 
# decomp_sample_pkl = open('decomp_sample_pkl.obj', 'wb') 
# pickle.dump(decomp_sample, decomp_sample_pkl)